In [ ]:
#;load data using with statement
with open("ca1_step1_input_data.txt",'r') as file: #data = file.read() 为read所有，read所有后不容易进行操作
    for i in range(7):
        header=file.readline()#不用header+=..也不用提前定义header，直接read进来一行一行
        print(header)#type:str, 不能header.readline()因为Str不能readline，file可以
    """
    #读一行
    line = file.readline()
    print(line)
    #读n行
    line = file.readlines()#读成list
    header = line[0:2] #这样的话header.split不可用因为split用于str
    print(header)
    """

In [7]:
value=[]
with open("ca1_step1_input_data.txt",'r') as file:
    for i in range(2):
        header=file.readline()
        if i==0:
            header=header.replace(';',',')
            header=header.replace('#',' ')
            header=header.replace('(s)',' ')
            header=header.replace('\t',' ')
            header=header.replace('\n',' ')
            header=header.replace(' ','')
            header=header.split(',')
            key=header#list
        elif i==1:
            header=header.replace(';',',')
            header=header.replace('#',' ')
            header=header.replace('(s)',' ')
            header=header.replace('\t',' ')
            header=header.replace('\n',' ')
            header=header.replace(' ','')
            header=header.split(',')#list of string
            for index in range(len(header)):#len(header)=5,header=['1000', '0.0200000000000000', '5.000000E-02', '1.000000E-01', '400']list
                try:
                    value += [int(header[index])]#int(0.0002)这里会报错，转到float
                except:
                    value += [float(header[index])]
                finally:
                    pass#after this, list of number
            for name,value in zip(key,value):#把key-value zip到一起成zip形式(不是tuple)（若想print则list(zip(..))），然后iterate
                #把vaAlue值赋给name,就像a=9一样赋值，这里不是字典
                #NB：exec(obj)执行一个object，exec 'print "Hello World"'>>Hello World
                #NB: f''(f""),used to format strings:name = "Eric",age = 74...f"Hello, {name}. You are {age}.">>'Hello, Eric. You are 74.'
                #NB:注意必须加{}
                exec(f"{name} = {value}")
            print(time_steps, time_step, radius, v_variance, N_particles)

1000 0.02 0.05 0.1 400


In [6]:
# parse main header
value=[]
dim1=[]#vx vy x y
dim2=[]#400 particles
dim3=[]#1000 snapshots
with open("ca1_step1_input_data.txt",'r') as file:
    for i in range(1000000):
        header=file.readline()
        if header == '':
            break
        if i==0:
            header=header.replace(';',',')
            header=header.replace('#',' ')
            header=header.replace('(s)',' ')
            header=header.replace('\t',' ')
            header=header.replace('\n',' ')
            header=header.replace(' ','')
            header=header.split(',')
            key=header#list
        elif i==1:
            header=header.replace(';',',')
            header=header.replace('#',' ')
            header=header.replace('(s)',' ')
            header=header.replace('\t',' ')
            header=header.replace('\n',' ')
            header=header.replace(' ','')
            header=header.split(',')#list of string
            for index in range(len(header)):#len(header)=5,header=['1000', '0.0200000000000000', '5.000000E-02', '1.000000E-01', '400']list
                try:
                    value += [int(header[index])]#int(0.0002)这里会报错，转到float
                except:
                    value += [float(header[index])]
                finally:
                    pass#after this, list of number
            for name,value in zip(key,value):#把key-value zip到一起成zip形式(不是tuple)（若想print则list(zip(..))），然后iterate
                #把vaAlue值赋给name,就像a=9一样赋值，这里不是字典
                #NB：exec(obj)执行一个object，exec 'print "Hello World"'>>Hello World
                #NB: f''(f""),used to format strings:name = "Eric",age = 74...f"Hello, {name}. You are {age}.">>'Hello, Eric. You are 74.'
                #NB:注意必须加{}
                exec(f"{name} = {value}")
            print(time_steps, time_step, radius, v_variance, N_particles)
        else:
            if '# time_step' in header:
                header=header.replace('\n','')
                dim3.append(dim2)
                dim2=[]
            elif '# x, y; v_x, v_y' in header:
                pass
            elif header == '\n':
                pass
            else:
                header=header.replace(',','')
                header=header.replace(';','')
                header=header.replace('\n','')
                header=header.split()
                dim1= [float(i) for i in header]#把header每个元素带入i，然后float(i)。i只是index，无实际意义                              
                dim2.append(dim1)
    dim3.append(dim2)   
    data=list(filter(None,dim3))#去除不要的
    #print(data)
    print(len(data), len(data[0]), len(data[0][0]), type(data[0][0][0]))

1000 0.02 0.05 0.1 400
1000 400 4 <class 'float'>


In [2]:
import numpy as np
Nd=np.array(data)
print(type(Nd))
print(Nd.dtype)
print(Nd.shape)

<class 'numpy.ndarray'>
float64
(1000, 400, 4)


In [3]:
R = Nd[0:, 0:, 0:2]
V = Nd[0:, 0:, 2:4]
R = np.swapaxes(R, 1, 2)
V = np.swapaxes(V, 1, 2)
print(R.shape)
print(V.shape)
print(type(R))

(1000, 2, 400)
(1000, 2, 400)
<class 'numpy.ndarray'>


In [4]:
import matplotlib.pyplot as plt
plt.scatter(R[0][0][:], R[0][1][:])
plt.axis([-1, 1,-1, 1],'equal')
plt.show()    

<Figure size 640x480 with 1 Axes>

In [8]:
with h5py.File("ca1_step1_input_data.hdf5", "w") as f:
    f.create_dataset("R", data=R)
    f.create_dataset("V", data=V)
    f.attrs[’time_steps’] = time_steps
    f.attrs[’time_step’] = time_step
    f.attrs[’radius’] = radius
    f.attrs[’v_variance’] = v_variance
    f.attrs[’N_particles’] = N_particles

SyntaxError: invalid character in identifier (<ipython-input-8-d881e5325470>, line 4)